In [1]:
import pandas as pd

In [2]:
listings_df = pd.read_csv("../csv_data/listings_full.csv.gz")

/Users/utbootcamp/.conda/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cols = ["host_is_superhost","host_since","host_response_time","host_response_rate","host_listings_count",
        "host_has_profile_pic","host_identity_verified","neighbourhood_cleansed","latitude",
        "longitude","is_location_exact","property_type","room_type","accommodates","bathrooms",
        "bedrooms","beds","bed_type","price","weekly_price","monthly_price","security_deposit",
        "cleaning_fee","guests_included","extra_people","minimum_nights","maximum_nights",
        "availability_30","availability_60","availability_90","availability_365","number_of_reviews",
        "number_of_reviews_ltm","review_scores_rating","review_scores_accuracy","review_scores_cleanliness"
        ,"review_scores_checkin","review_scores_communication","review_scores_location",
        "review_scores_value","instant_bookable","cancellation_policy","require_guest_profile_picture",
        "require_guest_phone_verification","calculated_host_listings_count",
        "calculated_host_listings_count_entire_homes","calculated_host_listings_count_private_rooms",
        "calculated_host_listings_count_shared_rooms","reviews_per_month"]
listings_df = listings_df[cols]

In [4]:
listings_df.rename(columns={"neighbourhood_cleansed": "zipcode"}, inplace=True)

In [5]:
listings_df = listings_df.query("host_is_superhost.notnull()", engine='python')

In [6]:
boolean_cols = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified","is_location_exact",
                "instant_bookable", "require_guest_profile_picture", "require_guest_phone_verification"]

for col in boolean_cols:
    listings_df[col] = listings_df[col].str.replace("t","1").str.replace("f","0").astype(int)

In [7]:
listings_df["host_response_rate"] = listings_df["host_response_rate"].str.replace("%","") \
                                    .astype(float).fillna(0) / 100

In [8]:
price_cols = ["price", "weekly_price", "monthly_price", "security_deposit", "cleaning_fee", "extra_people"]

for col in price_cols:
    listings_df[col] = listings_df[col].str.replace("$","").str.replace(",","").astype(float).fillna(0)

In [9]:
listings_df["total_hosting_years"] = 2019 - listings_df["host_since"].str.split("-") \
                                                               .str[0].fillna(2019).astype("int")
listings_df.drop(columns=["host_since"], inplace=True)

In [10]:
listings_df["host_listings_count"].fillna(1, inplace=True)

In [11]:
fill_na_cols = ["host_has_profile_pic", "host_identity_verified", "bathrooms", "bedrooms", "beds", 
                "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", 
                "review_scores_checkin", "review_scores_communication", "review_scores_location", 
                "review_scores_value", "reviews_per_month"]

for col in fill_na_cols:
    listings_df[col].fillna(0, inplace=True)

In [12]:
listings_df["host_response_time"].fillna("unknown", inplace=True)

In [13]:
zipcode_regions_df = pd.read_csv("../csv_data/airbnb_loc_regions.csv")[["zipcode", "region_by_zipcode"]]

In [14]:
zipcode_regions_df = zipcode_regions_df.drop_duplicates("zipcode") \
                        .query("zipcode.notnull()", engine='python') \
                        .reset_index(drop=True)

In [15]:
final_df = pd.merge(left=listings_df, right=zipcode_regions_df, how="inner", on="zipcode")

In [16]:
final_df.to_csv("../csv_data/listings_classification.csv", index=False)